In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data


def get_train_data():
    """得到训练数据，这里使用随机数生成训练数据，由此导致最终结果并不好"""

    def get_tensor_from_pd(dataframe_series) -> torch.Tensor:
        return torch.tensor(data=dataframe_series.values)

    import numpy as np
    import pandas as pd
    from sklearn import preprocessing
    # 生成训练数据x并做归一化后，构造成dataframe格式，再转换为tensor格式
    df = pd.DataFrame(data=preprocessing.MinMaxScaler().fit_transform(np.random.randint(0, 10, size=(2000, 300))))
    y = pd.Series(np.random.randint(0, 2, 2000))
    return get_tensor_from_pd(df).float(), get_tensor_from_pd(y).float()


class LstmAutoEncoder(nn.Module):
    def __init__(self, input_layer=300, hidden_layer=100, batch_size=20):
        super(LstmAutoEncoder, self).__init__()

        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.batch_size = batch_size
        self.encoder_lstm = nn.LSTM(self.input_layer, self.hidden_layer, batch_first=True)
        self.decoder_lstm = nn.LSTM(self.hidden_layer, self.input_layer, batch_first=True)

    def forward(self, input_x):
        input_x = input_x.view(len(input_x), 1, -1)
        # encoder
        encoder_lstm, (n, c) = self.encoder_lstm(input_x,
                                                 (torch.zeros(1, self.batch_size, self.hidden_layer),
                                                  torch.zeros(1, self.batch_size, self.hidden_layer)))
        # decoder
        decoder_lstm, (n, c) = self.decoder_lstm(encoder_lstm,
                                                 (torch.zeros(1, self.batch_size, self.input_layer),
                                                  torch.zeros(1, self.batch_size, self.input_layer)))
        return decoder_lstm.squeeze()


class LstmFcAutoEncoder(nn.Module):
    def __init__(self, input_layer=300, hidden_layer=100, batch_size=20):
        super(LstmFcAutoEncoder, self).__init__()

        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.batch_size = batch_size

        self.encoder_lstm = nn.LSTM(self.input_layer, self.hidden_layer, batch_first=True)
        self.encoder_fc = nn.Linear(self.hidden_layer, self.hidden_layer)
        self.decoder_lstm = nn.LSTM(self.hidden_layer, self.input_layer, batch_first=True)
        self.decoder_fc = nn.Linear(self.hidden_layer, self.hidden_layer)
        self.relu = nn.ReLU()

    def forward(self, input_x):
        input_x = input_x.view(len(input_x), 1, -1)
        # encoder
        encoder_lstm, (n, c) = self.encoder_lstm(input_x,
                                                 # shape: (n_layers, batch, hidden_size)
                                                 (torch.zeros(1, self.batch_size, self.hidden_layer),
                                                  torch.zeros(1, self.batch_size, self.hidden_layer)))
        encoder_fc = self.encoder_fc(encoder_lstm)
        encoder_out = self.relu(encoder_fc)
        # decoder
        decoder_fc = self.relu(self.decoder_fc(encoder_out))
        decoder_lstm, (n, c) = self.decoder_lstm(decoder_fc,
                                                 (torch.zeros(1, 20, self.input_layer),
                                                  torch.zeros(1, 20, self.input_layer)))
        return decoder_lstm.squeeze()


if __name__ == '__main__':
    # 得到数据
    x, y = get_train_data()
    train_loader = Data.DataLoader(
        dataset=Data.TensorDataset(x, y),  # 封装进Data.TensorDataset()类的数据，可以为任意维度
        batch_size=20,  # 每块的大小
        shuffle=True,  # 要不要打乱数据 (打乱比较好)
        num_workers=2,  # 多进程（multiprocess）来读数据
    )
    # 建模三件套：loss，优化，epochs
    model = LstmAutoEncoder()  # lstm
    # model = LstmFcAutoEncoder()  # lstm+fc模型
    loss_function = nn.MSELoss()  # loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 优化器
    epochs = 150
    # 开始训练
    model.train()
    for i in range(epochs):
        for seq, labels in train_loader:
            optimizer.zero_grad()
            y_pred = model(seq).squeeze()  # 压缩维度：得到输出，并将维度为1的去除
            single_loss = loss_function(y_pred, seq)
            # 若想要获得类别，二分类问题使用四舍五入的方法即可：print(torch.round(y_pred))
            single_loss.backward()
            optimizer.step()
            print("Train Step:", i, " loss: ", single_loss)
		# 每20次，输出一次前20个的结果，对比一下效果
        if i % 20 == 0:
            test_data = x[:20]
            y_pred = model(test_data).squeeze()  # 压缩维度：得到输出，并将维度为1的去除
            print("TEST: ", test_data)
            print("PRED: ", y_pred)
            print("LOSS: ", loss_function(y_pred, test_data))



Train Step: 0  loss:  tensor(0.3568, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3488, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3400, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3368, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3310, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3289, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3127, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.3010, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.2888, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.2764, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.2563, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.2405, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.2219, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.2133, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  tensor(0.1943, grad_fn=<MseLossBackward0>)
Train Step: 0  loss:  ten

Train Step: 1  loss:  tensor(0.1013, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1014, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1030, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1021, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1003, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1017, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1031, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1023, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1028, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1020, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1029, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1010, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1009, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1025, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  tensor(0.1010, grad_fn=<MseLossBackward0>)
Train Step: 1  loss:  ten

Train Step: 2  loss:  tensor(0.1024, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1005, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1036, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1015, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1010, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1041, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1028, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1027, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1012, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1031, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1022, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1013, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1017, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1051, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  tensor(0.1020, grad_fn=<MseLossBackward0>)
Train Step: 2  loss:  ten

Train Step: 3  loss:  tensor(0.1019, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1019, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1009, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.0994, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1007, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1011, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.0998, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1021, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1002, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1021, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1009, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1008, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1009, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1011, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  tensor(0.1015, grad_fn=<MseLossBackward0>)
Train Step: 3  loss:  ten

Train Step: 5  loss:  tensor(0.1005, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0995, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1009, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1011, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1002, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1005, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0987, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1038, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0993, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0988, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1023, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0988, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0994, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.0996, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  tensor(0.1032, grad_fn=<MseLossBackward0>)
Train Step: 5  loss:  ten

...

Train Step: 143  loss:  tensor(0.0565, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0571, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0571, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0562, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0560, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0572, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0585, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0568, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0572, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0566, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0564, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0557, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0585, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0571, grad_fn=<MseLossBackward0>)
Train Step: 143  loss:  tensor(0.0564, grad_fn=<MseLossBackwar

Train Step: 144  loss:  tensor(0.0580, grad_fn=<MseLossBackward0>)
Train Step: 144  loss:  tensor(0.0596, grad_fn=<MseLossBackward0>)
Train Step: 144  loss:  tensor(0.0577, grad_fn=<MseLossBackward0>)
Train Step: 144  loss:  tensor(0.0560, grad_fn=<MseLossBackward0>)
Train Step: 144  loss:  tensor(0.0563, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0581, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0581, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0565, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0553, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0575, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0551, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0559, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0559, grad_fn=<MseLossBackward0>)
Train Step: 145  loss:  tensor(0.0585, grad_fn=<MseLossBackwar

Train Step: 146  loss:  tensor(0.0566, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0574, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0578, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0564, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0584, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0588, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0578, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0566, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0567, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0572, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0557, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0576, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0574, grad_fn=<MseLossBackward0>)
Train Step: 146  loss:  tensor(0.0557, grad_fn=<MseLossBackwar

Train Step: 147  loss:  tensor(0.0566, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0581, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0577, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0568, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0559, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0582, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0576, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0562, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0571, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0553, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0584, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0589, grad_fn=<MseLossBackward0>)
Train Step: 147  loss:  tensor(0.0580, grad_fn=<MseLossBackwar

Train Step: 148  loss:  tensor(0.0597, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0566, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0575, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0563, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0575, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0575, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0574, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0570, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0576, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0574, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0561, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0574, grad_fn=<MseLossBackward0>)
Train Step: 148  loss:  tensor(0.0570, grad_fn=<MseLossBackwar